# Assignment 2
##### Joseph High
##### UNI: jph2185

In [1]:
import requests
import json

In [2]:
#My api_key
with open("./places_api_key.txt",'r') as f:
    api_key = f.read().strip()

In [5]:
def get_businesses(query,api_key,min_rating=0.0,max_results=30):
        response_data = '' #string to store JSON data
        url='https://maps.googleapis.com/maps/api/place/textsearch/json?query=%s&key=%s' % (query,api_key)

        try:
            response = requests.get(url)
            if not response.status_code == 200:
                print("HTTP error",response.status_code)
            else:
                try:
                    response_data = response.json()
                except:
                    print("Response not in valid JSON format")
        except:
            print("Something went wrong with requests.get")

        ## If applicable, obtaining data on next page(s) (pagination)
        while len(response_data['results']) <= max_results:
            if max_results > 20 and response_data['next_page_token']:
                new_resp_data = ''
                next_page_token = response_data['next_page_token']
                new_url="https://maps.googleapis.com/maps/api/place/textsearch/json?pagetoken=%s&key=%s" % (next_page_token,api_key)

                ## Pausing script to give time for next page:
                response = requests.get(new_url)
                while response.json().get('status') != 'OK':
                    from time import sleep
                    from random import random
                    sleep(random())
                    response = requests.get(new_url)
                    new_resp_data = response.json()

                response_data['results'].extend(new_resp_data['results'])  #augment results with new results on next page
            else:
                break
        
        businesses=[]  #define empty list to store restaurants that fit the input parameters
        ## Only display restaurants with ratings above the min_rating argument
        for biz in response_data['results']:
            if biz['rating'] >= min_rating:
                try: 
                    biz['price_level']
                except:
                    biz['price_level'] = None
                businesses.append((biz['name'],biz['formatted_address'],biz['opening_hours'],biz['price_level'],biz['rating']))
        
        restaurants=[]  #using new list to ensure the correct number of restaurants are returned
        if len(businesses) > max_results:
            restaurants = businesses[:max_results]
        else:
            restaurants = businesses
        return restaurants

In [6]:
get_businesses('restaurants near Columbia University', api_key, min_rating=3.5, max_results=60)

TypeError: string indices must be integers

In [ ]:
len(get_businesses('restaurants near Columbia University', api_key, min_rating=3.5, max_results=60))